# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
0,0,longyearbyen,78.22,15.64,-11.60,-11.60,57,0,2.08
1,1,ushuaia,-54.80,-68.30,40.06,40.06,82,100,5.44
2,2,esperance,-33.87,121.90,66.40,66.40,80,86,3.38
3,3,shache,38.42,77.24,20.82,20.82,72,65,2.17
4,4,sovetskiy,61.36,63.58,-6.72,-6.72,98,44,3.47


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
coord = weather_df[['Latitude', 'Longitude']]

In [4]:

figure_layout = {'width' : '800px', 'height' : '600px', 'border' : '1px solid black', 'padding' : '1px',
                 'margin' : '0 auto 0 auto'}
fig = gmaps.figure(layout = figure_layout)
humidity = gmaps.heatmap_layer(coord, weights = weather_df['Humidity (%)'], dissipating = False, max_intensity = 100, point_radius = 5)
fig.add_layer(humidity)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_cities_df = weather_df.loc[(weather_df['Max Temperature (F)'] > 70) & (weather_df['Max Temperature (F)'] < 80) &
                                    (weather_df['Wind Speed (MPH)'] < 10) & (weather_df['Cloudiness (%)'] == 0)]
vacation_cities_df

,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
31,31,cockburn town,21.46,-71.14,76.05,76.05,71,0,9.28
100,100,arraial do cabo,-22.97,-42.03,75.79,75.79,83,0,9.40
173,173,marawi,8.00,124.29,77.00,77.00,90,0,4.00
252,252,buchanan,5.88,-10.05,76.21,76.21,94,0,0.67
273,273,niteroi,-22.88,-43.10,75.20,75.20,81,0,3.22
320,320,rosario do sul,-30.26,-54.91,78.21,78.21,40,0,4.41
356,356,aranos,-24.13,19.12,73.11,73.11,15,0,7.52
363,363,lakes entrance,-37.88,147.98,71.94,73.00,68,0,1.01
435,435,puerto rico,-26.80,-55.02,75.07,75.07,47,0,3.89


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

In [7]:
hotels = []
countrys = []
for lat, lon in zip(vacation_cities_df['Latitude'], vacation_cities_df['Longitude']):
    
    query_url = f'{base_url}location={lat},{lon}&radius=5000&type=hotel&key={g_key}'
    response = requests.get(query_url).json()
    
    try:
        hotels.append(response['results'][1]['name'])
        countrys.append(response['results'][1]['plus_code']['compound_code'])
    except TypeError:
        print('failed')
        
countrys


['FV53+R2 Cockburn Town, Turks & Caicos Islands',
 '2XCG+RF Macedônia, Arraial do Cabo - State of Rio de Janeiro, Brazil',
 '272M+FQ Marawi City, Lanao del Sur, Philippines',
 'VX52+84 Arcelor Mittal Liberia Concession Area, Buchanan, Liberia',
 '3VWH+P3 Ingá, Niterói - State of Rio de Janeiro, Brazil',
 'Q33M+G5 Rosário do Sul, State of Rio Grande do Sul, Brazil',
 'V459+22 Aranos, Namibia',
 '4X9F+2Q Lakes Entrance, Victoria, Australia',
 '52H2+F2 San Alberto, Misiones Province, Argentina']

In [8]:
vaca_country = []
for country in countrys:
    split_country = country.split(',')
    vaca_country.append(split_country[-1])
vaca_country

[' Turks & Caicos Islands',
 ' Brazil',
 ' Philippines',
 ' Liberia',
 ' Brazil',
 ' Brazil',
 ' Namibia',
 ' Australia',
 ' Argentina']

In [9]:
vacation_cities_df['Hotel Name'] = hotels
vacation_cities_df['Country'] = vaca_country
vacation_cities_df
                                        

C:\Users\hurle\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\hurle\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Hotel Name,Country
31,31,cockburn town,21.46,-71.14,76.05,76.05,71,0,9.28,Osprey Beach Hotel,Turks & Caicos Islands
100,100,arraial do cabo,-22.97,-42.03,75.79,75.79,83,0,9.40,Pousada Porto Praia,Brazil
173,173,marawi,8.00,124.29,77.00,77.00,90,0,4.00,Marawi City Hall,Philippines
252,252,buchanan,5.88,-10.05,76.21,76.21,94,0,0.67,Port of Buchanan,Liberia
273,273,niteroi,-22.88,-43.10,75.20,75.20,81,0,3.22,OYO Viareggio Hotel - Niterói,Brazil
320,320,rosario do sul,-30.26,-54.91,78.21,78.21,40,0,4.41,Hotel Areias Brancas,Brazil
356,356,aranos,-24.13,19.12,73.11,73.11,15,0,7.52,Ley-Lia Guest House,Namibia
363,363,lakes entrance,-37.88,147.98,71.94,73.00,68,0,1.01,The Esplanade Resort & Spa,Australia
435,435,puerto rico,-26.80,-55.02,75.07,75.07,47,0,3.89,Hotel Cabañas del Prado,Argentina


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_cities_df.iterrows()]
locations = vacation_cities_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations,  fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content =  hotel_info)
fig.add_layer(hotel_layer)
fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…